In [3]:
import numpy as np
import pandas as pd


In [4]:
import os
for dirname, _, filenames in os.walk('/content/drive/MyDrive/Demoday_Project/Dataset/BBC_news/BBC_News_Summary/BBC_News_Summary'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/content/drive/MyDrive/Demoday_Project/Dataset/BBC_news/BBC_News_Summary/BBC_News_Summary/News Articles/business/004.txt
/content/drive/MyDrive/Demoday_Project/Dataset/BBC_news/BBC_News_Summary/BBC_News_Summary/News Articles/business/005.txt
/content/drive/MyDrive/Demoday_Project/Dataset/BBC_news/BBC_News_Summary/BBC_News_Summary/News Articles/business/003.txt
/content/drive/MyDrive/Demoday_Project/Dataset/BBC_news/BBC_News_Summary/BBC_News_Summary/News Articles/business/001.txt
/content/drive/MyDrive/Demoday_Project/Dataset/BBC_news/BBC_News_Summary/BBC_News_Summary/News Articles/business/002.txt
/content/drive/MyDrive/Demoday_Project/Dataset/BBC_news/BBC_News_Summary/BBC_News_Summary/News Articles/business/008.txt
/content/drive/MyDrive/Demoday_Project/Dataset/BBC_news/BBC_News_Summary/BBC_News_Summary/News Articles/business/028.txt
/content/drive/MyDrive/Demoday_Project/Dataset/BBC_news/BBC_News_Summary/BBC_News_Summary/News Articles/business/033.txt
/content/drive/MyDrive/Demoday_P

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
import warnings
warnings.filterwarnings("ignore")

In [7]:
!pip install transformers
!pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 73.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 120.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 68.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [8]:
import os
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import tensorflow as tf
import tensorflow.keras

In [9]:
test_split = 0.2
batch_size = 64
epochs = 2
learning_rate = 1e-4

In [10]:
#sorted function
dataset_path = '/content/drive/MyDrive/Demoday_Project/Dataset/BBC_news/BBC_News_Summary/BBC_News_Summary'
articles_path = sorted(glob.glob(os.path.join(dataset_path, "News Articles/*/*.txt")))
summaries_path = sorted(glob.glob(os.path.join(dataset_path, "Summaries/*/*.txt")))
print(articles_path[3], summaries_path[3])


/content/drive/MyDrive/Demoday_Project/Dataset/BBC_news/BBC_News_Summary/BBC_News_Summary/News Articles/business/004.txt /content/drive/MyDrive/Demoday_Project/Dataset/BBC_news/BBC_News_Summary/BBC_News_Summary/Summaries/business/004.txt


In [11]:
articles = []
summaries = []

for article_path, summary_path in zip(articles_path, summaries_path):
    with open(article_path, "r", encoding="ISO-8859-1") as article_file:
        article_text = article_file.read()
        articles.append(article_text)

    with open(summary_path, "r", encoding="ISO-8859-1") as summary_file:
        summary_text = summary_file.read()
        summaries.append(summary_text)

In [12]:
categories_list = os.listdir(os.path.join(dataset_path, 'News Articles'))
categories = []
def print_file_data():
    for category in categories_list:
        article_paths = glob.glob(os.path.join(dataset_path, "News Articles", category, "*.txt"))
        summary_paths = glob.glob(os.path.join(dataset_path, "Summaries", category, "*.txt"))

        for i in article_paths:
            categories.append(category)

        if len(article_paths) != len(summary_paths):
            print("Length of dataset not equal for subdirectory :", category)
        else:
            print(f"Found {len(article_paths)} articles and {len(summary_paths)} summaries in the subdirectory : {category}")

print_file_data()

Found 510 articles and 510 summaries in the subdirectory : business
Found 401 articles and 401 summaries in the subdirectory : tech
Found 386 articles and 386 summaries in the subdirectory : entertainment
Found 511 articles and 511 summaries in the subdirectory : sport
Found 417 articles and 417 summaries in the subdirectory : politics


In [13]:
articles[2]

'Yukos unit buyer faces loan claim\n\nThe owners of embattled Russian oil giant Yukos are to ask the buyer of its former production unit to pay back a $900m (Â£479m) loan.\n\nState-owned Rosneft bought the Yugansk unit for $9.3bn in a sale forced by Russia to part settle a $27.5bn tax claim against Yukos. Yukos\' owner Menatep Group says it will ask Rosneft to repay a loan that Yugansk had secured on its assets. Rosneft already faces a similar $540m repayment demand from foreign banks. Legal experts said Rosneft\'s purchase of Yugansk would include such obligations. "The pledged assets are with Rosneft, so it will have to pay real money to the creditors to avoid seizure of Yugansk assets," said Moscow-based US lawyer Jamie Firestone, who is not connected to the case. Menatep Group\'s managing director Tim Osborne told the Reuters news agency: "If they default, we will fight them where the rule of law exists under the international arbitration clauses of the credit."\n\nRosneft official

In [14]:
summaries[2]

'Yukos\' owner Menatep Group says it will ask Rosneft to repay a loan that Yugansk had secured on its assets.State-owned Rosneft bought the Yugansk unit for $9.3bn in a sale forced by Russia to part settle a $27.5bn tax claim against Yukos.The sale went ahead in December and Yugansk was sold to a little-known shell company which in turn was bought by Rosneft.But the company has said it intends to take action against Menatep to recover some of the tax claims and debts owed by Yugansk."The pledged assets are with Rosneft, so it will have to pay real money to the creditors to avoid seizure of Yugansk assets," said Moscow-based US lawyer Jamie Firestone, who is not connected to the case.'

In [15]:
df = pd.DataFrame({
    'Articles' : articles,
    'Summaries' : summaries,
    #'Categories' : categories
})

df.head()

,Articles,Summaries
0,Ad sales boost Time Warner profit\n\nQuarterly...,TimeWarner said fourth quarter sales rose 2% t...
1,Dollar gains on Greenspan speech\n\nThe dollar...,The dollar has hit its highest level against t...
2,Yukos unit buyer faces loan claim\n\nThe owner...,Yukos' owner Menatep Group says it will ask Ro...
3,High fuel prices hit BA's profits\n\nBritish A...,"Rod Eddington, BA's chief executive, said the ..."
4,Pernod takeover talk lifts Domecq\n\nShares in...,Pernod has reduced the debt it took on to fund...


In [16]:
#category = df.groupby("Categories").size()
#print(category)
#plt.figure(figsize=(10,6))
#sns.barplot(x=category.index, y=category)
#plt.title("Category wise distribution of Articles")
#plt.show()

In [17]:
df['Article Length'] = df["Articles"].apply(lambda x: len(x.split()))
df['Summary Length'] = df["Summaries"].apply(lambda x: len(x.split()))

In [18]:
df

,Articles,Summaries,Article Length,Summary Length
0,Ad sales boost Time Warner profit\n\nQuarterly...,TimeWarner said fourth quarter sales rose 2% t...,421,134
1,Dollar gains on Greenspan speech\n\nThe dollar...,The dollar has hit its highest level against t...,384,158
2,Yukos unit buyer faces loan claim\n\nThe owner...,Yukos' owner Menatep Group says it will ask Ro...,264,121
3,High fuel prices hit BA's profits\n\nBritish A...,"Rod Eddington, BA's chief executive, said the ...",406,197
4,Pernod takeover talk lifts Domecq\n\nShares in...,Pernod has reduced the debt it took on to fund...,265,106
...,...,...,...,...
2220,BT program to beat dialler scams\n\nBT is intr...,BT is introducing two initiatives to help beat...,420,186
2221,Spam e-mails tempt net shoppers\n\nComputer us...,A third of them read unsolicited junk e-mail a...,357,130
2222,Be careful how you code\n\nA new European dire...,This goes to the heart of the European project...,1050,475
2223,US cyber security chief resigns\n\nThe man mak...,Amit Yoran was director of the National Cyber ...,379,137


In [19]:
from transformers import AutoTokenizer
import transformers
import torch

model = "tiiuae/falcon-7b-instruct"

tokenizer = AutoTokenizer.from_pretrained(model)
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
)

A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b-instruct:
- configuration_RW.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


ImportError: ignored

In [ ]:
#tokenizer = BartTokenizer.from_pretrained("facebook/bart-large")
#model = TFBartForConditionalGeneration.from_pretrained("facebook/bart-large")

All model checkpoint layers were used when initializing TFBartForConditionalGeneration.

All the layers of TFBartForConditionalGeneration were initialized from the model checkpoint at facebook/bart-large.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBartForConditionalGeneration for predictions without further training.


In [ ]:
#tokenizer

BartTokenizer(name_or_path='facebook/bart-large', vocab_size=50265, model_max_length=1024, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'eos_token': AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'unk_token': AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'sep_token': AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'pad_token': AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'cls_token': AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'mask_token': AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=True)}, clean_up_tokenization_spaces=True)

In [ ]:
#model

In [ ]:
filters = '[!"#$%&()*+,-/:;=?@[\\]^_`{|}~\t\n]'
df["Articles"] = df["Articles"].str.replace('.', ' ', regex=True)
df['Articles'] = df["Articles"].str.replace(filters, '', regex=True)
df["Summaries"] = df["Summaries"].str.replace('.', ' ', regex=True)
df['Summaries'] = df["Summaries"].str.replace(filters, '', regex=True)

In [ ]:
df["Articles"][3]

In [ ]:
from sklearn.model_selection import train_test_split
X = df["Articles"]
Y = df["Summaries"]

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
train_df = pd.concat([X_train, Y_train], axis=1)
test_df = pd.concat([X_test, Y_test], axis=1)

In [ ]:
train_df

In [ ]:
print((X_train.shape, Y_train.shape), (X_test.shape, Y_test.shape))

In [ ]:
from transformers import InputExample, InputFeatures
def convert_data_to_examples(train, test, data, label):
    train_input_examples = train.apply(lambda x: InputExample(
        guid = None,
        text_a = x[data],
        text_b = None,
        label = x[label]
    ), axis=1)

    test_input_examples = test.apply(lambda x: InputExample(
        guid = None,
        text_a = x[data],
        text_b = None,
        label = x[label]
    ), axis=1)

    return train_input_examples, test_input_examples

In [ ]:
train_input_examples, test_input_examples = convert_data_to_examples(train_df, test_df, "Articles", "Summaries")

In [ ]:
train_input_examples

In [ ]:
test_input_examples

In [ ]:
train_input_examples[6]

In [ ]:
#'Run this cell to see how to tokenizer.encode_plus works'


# input_dict = tokenizer.encode_plus(
#             list(train_input_examples[6].text_a),
#             add_special_tokens = True, # adds <start> and <end> tokens
#             max_length = 128, # defining max_len for padding
#             return_token_type_ids = True, # return token_type_ids
#             pad_to_max_length = True, # pads the sentences shorter than max_len
#             return_attention_mask = True, # return attention mask (tells model if the padded token is actual sentence or just added for dimension coorection of instance)
#             truncation = True
#         )
# input_dict

In [ ]:
# Tokenizing examples and converting into tensorflow dataset
def convert_examples_to_tf_dataset(examples, tokenizer, max_len=1024):
    features = [] #     Array to store the features

    for example in examples:
        input_dict = tokenizer.encode_plus(
            list(example.text_a),
            add_special_tokens = True, # adds <start> and <end> tokens
            max_length = max_len, # defining max_len for padding
            return_token_type_ids = True, # return token_type_ids
            pad_to_max_length = True, # pads the sentences shorter than max_len
            return_attention_mask = True, # return attention mask (tells model if the padded token is actual sentence or just added for dimension coorection of instance)
            truncation = True
        )
#       Returning all the params
        input_ids, attention_masks = (input_dict["input_ids"], input_dict["attention_mask"])



#         Same processing for labels
        label_dict = tokenizer.encode_plus(
            list(example.label),
            add_special_tokens = True, # adds <start> and <end> tokens
            max_length = max_len, # defining max_len for padding
            return_token_type_ids = True, # return token_type_ids
            pad_to_max_length = True, # pads the sentences shorter than max_len
            return_attention_mask = True, # return attention mask (tells model if the padded token is actual sentence or just added for dimension coorection of instance)
            truncation = True
        )

        label_ids = label_dict["input_ids"]

#       Adding these params in the features array to feed into the model later
        features.append(
            InputFeatures(
                input_ids = input_ids, attention_mask = attention_masks, label=label_ids
            )
        )


        def gen():
            for f in features:
                yield(
                    {
                        'input_ids':f.input_ids,
                        'attention_mask':f.attention_mask,
                    },
                    f.label[0],
                )



#     Returning data as tensroflow dataset for easy input in our model
    return tf.data.Dataset.from_generator(
        gen,
        ({"input_ids": tf.int32, "attention_mask": tf.int32}, tf.int32),
        (
            {
                "input_ids":tf.TensorShape([None]),
                "attention_mask":tf.TensorShape([None]),
            },
            tf.TensorShape([]),
        ),
    )


In [ ]:
# Preparing the dataset
train_data = convert_examples_to_tf_dataset(list(train_input_examples), tokenizer)
test_data = convert_examples_to_tf_dataset(list(train_input_examples), tokenizer)

In [ ]:
train_data.take(0)

In [ ]:
test_data

In [ ]:
# Preparing shuffles batches
train_data = train_data.shuffle(100).batch(32)
test_data = test_data.batch(32)

In [ ]:
train_data

In [ ]:
test_data

In [ ]:
from keras.optimizers import Adam
from keras.losses import SparseCategoricalCrossentropy
from keras.metrics import SparseCategoricalAccuracy
model.compile(Adam(learning_rate=learning_rate),loss=SparseCategoricalCrossentropy(from_logits=True),
              metrics=[SparseCategoricalAccuracy('accuracy')])

In [ ]:
sequences = pipeline(
   "Write a poem about Valencia.",
    max_length=200,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")